In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [111]:
# Especifica la ruta al archivo CSV
path = r'C:\\Users\\joset\\OneDrive\\Documentos\\Master Big Data UCM\\18-TFN\NaturgyProject\\data'
# Lee el archivo CSV en un DataFrame de pandas
AEMET_Previsiones_original = pd.read_csv(path + '\\Originales\\AEMET_Previsiones.csv', encoding = 'latin1')
AEMET_Realidad_original = pd.read_csv(path + '\\Originales\\AEMET_Realidad.csv', encoding = 'latin1')

In [112]:
AEMET_Realidad = AEMET_Realidad_original.copy()
AEMET_Previsiones = AEMET_Previsiones_original.copy()

In [113]:
# Crear una nueva columna FECHA agrupando DIA/MES/ANYO como en el anterior dataframe

AEMET_Realidad['DIA'] = AEMET_Realidad['DIA'].astype(str).str.zfill(2)
AEMET_Realidad['MES'] = AEMET_Realidad['MES'].astype(str).str.zfill(2)

AEMET_Realidad['FECHA'] = AEMET_Realidad['DIA'] + '/' + AEMET_Realidad['MES'] + '/' + AEMET_Realidad['ANYO'].astype(str)
AEMET_Realidad['FECHA'] = pd.to_datetime(AEMET_Realidad['FECHA'], format='%d/%m/%Y')

AEMET_Realidad.head()

,INDICATIVO,ANYO,MES,DIA,NOMBRE,ALTITUD,NOM_PROV,LONGITUD,LATITUD,MAX,MIN,FECHA
0,0016A,2009,01,01,REUS/AEROPUERTO,71.0,TARRAGONA,110441.0,410859.0,13.0,4.2,2009-01-01
1,0016A,2009,01,02,REUS/AEROPUERTO,71.0,TARRAGONA,110441.0,410859.0,11.3,4.2,2009-01-02
2,0016A,2009,01,03,REUS/AEROPUERTO,71.0,TARRAGONA,110441.0,410859.0,10.6,6.1,2009-01-03
3,0016A,2009,01,04,REUS/AEROPUERTO,71.0,TARRAGONA,110441.0,410859.0,11.3,-0.8,2009-01-04
4,0016A,2009,01,05,REUS/AEROPUERTO,71.0,TARRAGONA,110441.0,410859.0,10.6,-3.1,2009-01-05


In [114]:
print(AEMET_Realidad.dtypes)

INDICATIVO            object
ANYO                   int64
MES                   object
DIA                   object
NOMBRE                object
ALTITUD              float64
NOM_PROV              object
LONGITUD             float64
LATITUD              float64
MAX                  float64
MIN                  float64
FECHA         datetime64[ns]
dtype: object


In [115]:
AEMET_Realidad[['MES', 'DIA']] = AEMET_Realidad[['MES', 'DIA']].astype(int)

In [116]:
# Las filas de 3168C son duplicados de 3168D. 3168C solo llega hasta 2014
AEMET_Realidad = AEMET_Realidad[AEMET_Realidad['INDICATIVO'] != '3168C']

# 1249I No tiene valores despúes de 2014 posiblemente sea el anterior a 1249x porque hay meses de 2014 que parece que se repiten las mediciones.
# Como vamos a utilizar a partir de 2016 no hay problema
AEMET_Realidad = AEMET_Realidad[AEMET_Realidad['INDICATIVO'] != '1249I']

# 1111 Parece que tiene valores duplicados con 1111X y solo llega hasta 2014
AEMET_Realidad = AEMET_Realidad[AEMET_Realidad['INDICATIVO'] != '1111']

# 8416 Parece que tiene valores duplicados con 8416Y y solo llega hasta 2014
AEMET_Realidad = AEMET_Realidad[AEMET_Realidad['INDICATIVO'] != '8416']

# 9434 Parece que tiene valores duplicados con 9434P y solo llega hasta 2014
AEMET_Realidad = AEMET_Realidad[AEMET_Realidad['INDICATIVO'] != '9434']



In [117]:
# Las filas con el INDICATIVO 1387 y LATITUD 433700 correspondientes a mes de marzo del 16 están 'duplicadas'
AEMET_Realidad = AEMET_Realidad[AEMET_Realidad['LATITUD'] != 433700]

In [118]:
# Eliminar las filas donde INDICATIVO es 1387E, LONGITUD es 822192 y LATITUD es nulo. Son duplicidades en fechas detectadas en 2015
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1387E') &
                                                    (AEMET_Realidad['LONGITUD'] == 822192) &
                                                    (AEMET_Realidad['LATITUD'].isnull())].index)

In [119]:
# El observatorio 8178D tiene valores duplicados en octubre del 22 cuando cambia la lat y long
# primero hay que borrar los duplicados (que tienen max y min varcia) y luego aunar las coordenadas
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '8178D') &
                                                    (AEMET_Realidad['LONGITUD'] == 151462) &
                                                    (AEMET_Realidad['LATITUD'] == 390021) &
                                                    (AEMET_Realidad['ANYO'] == 2022) &
                                                    (AEMET_Realidad['MES'] == 10) &
                                                    (AEMET_Realidad['MAX'].isnull())].index)

In [120]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '8178D', ['LATITUD', 'LONGITUD']] = [390021, 151462]

In [121]:
# El observatorio 8175 tiene valores duplicados en diciembre del 16 cuando cambia la lat y long
# primero hay que borrar los duplicados y luego aunar las coordenadas
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '8175') &
                                                    (AEMET_Realidad['LONGITUD'] == 151222) &
                                                    (AEMET_Realidad['LATITUD'] == 385715) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 12)].index)

In [122]:
# El observatorio 8175 tiene valores duplicados en octubre del 16 cuando cambia la lat y long
# primero hay que borrar los duplicados y luego aunar las coordenadas
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '8175') &
                                                    (AEMET_Realidad['LONGITUD'] == 151452) &
                                                    (AEMET_Realidad['LATITUD'] == 385706) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 10)].index)

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '8175') &
                                                    (AEMET_Realidad['LONGITUD'] == 151222) &
                                                    (AEMET_Realidad['LATITUD'] == 385715) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 9)].index)

In [123]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '8175', ['LATITUD', 'LONGITUD']] = [385715, 151232]

In [124]:
dias = [8,9,10,11,12]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '2444') &
                                                    (AEMET_Realidad['LONGITUD'] == 440472) &
                                                    (AEMET_Realidad['LATITUD'] == 403933) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['DIA'].isin(dias)) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [125]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '2444', ['LATITUD', 'LONGITUD']] = [403933, 440472]

In [126]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '0201D', ['LATITUD', 'LONGITUD']] = [412326, 212001]

In [127]:
dias = [8,9,10,11,12,13]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '2331') &
                                                    (AEMET_Realidad['LONGITUD'] == 337172) &
                                                    (AEMET_Realidad['LATITUD'] == 422125) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['DIA'].isin(dias)) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [128]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '2331', ['LATITUD', 'LONGITUD']] = [422125, 337172]

In [129]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '1082', ['LATITUD', 'LONGITUD']] = [431753, 254232]

In [130]:
dias = [11,12,13,14,15,16]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'BILBAO/AEROPUERTO') &
                                                    (AEMET_Realidad['INDICATIVO'] == '1082') &
                                                    (AEMET_Realidad['LONGITUD'] == 254232) &
                                                    (AEMET_Realidad['LATITUD'] == 431753) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['DIA'].isin(dias)) &
                                                    (AEMET_Realidad['MES'] == 5)].index)

In [131]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '1082', 'NOMBRE'] = 'BILBAO AEROPUERTO'

In [132]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1428') &
                                                    (AEMET_Realidad['LONGITUD'] == 824382) &
                                                    (AEMET_Realidad['LATITUD'].isnull())].index)

In [133]:
mes = [3,4,5,7,8,9,10]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1495') &
                                                    (AEMET_Realidad['LONGITUD'] == 837262) &
                                                    (AEMET_Realidad['LATITUD'].isnull()) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['MES'].isin(mes))].index)

In [134]:
mes = [1,2,4,5,6,7,9,11,12]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1495') &
                                                    (AEMET_Realidad['LONGITUD'] == 837262) &
                                                    (AEMET_Realidad['LATITUD'].isnull()) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'].isin(mes))].index)

In [135]:
dias = [1,2,3,4,5]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1428') &
                                                    (AEMET_Realidad['NOMBRE'] == 'SANTIAGO DE COMPOSTELA/LAVACOLLA') &
                                                    (AEMET_Realidad['LONGITUD'] == 824382) &
                                                    (AEMET_Realidad['LATITUD'] == 425317) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 5) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

dias1 = [14,15,16,17,18,19,30]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1428') &
                                                    (AEMET_Realidad['NOMBRE'] == 'SANTIAGO DE COMPOSTELA/LAVACOLLA') &
                                                    (AEMET_Realidad['LONGITUD'] == 824382) &
                                                    (AEMET_Realidad['LATITUD'] == 425317) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 4) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

dias2 = [19,20,21,22,23,24]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1428') &
                                                    (AEMET_Realidad['NOMBRE'] == 'SANTIAGO DE COMPOSTELA/LAVACOLLA') &
                                                    (AEMET_Realidad['LONGITUD'] == 824382) &
                                                    (AEMET_Realidad['LATITUD'] == 425317) &
                                                    (AEMET_Realidad['ANYO'] == 2024) &
                                                    (AEMET_Realidad['MES'] == 6) &
                                                    (AEMET_Realidad['DIA'].isin(dias2))].index)

In [136]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '1428', 'NOMBRE'] = 'SANTIAGO DE COMPOSTELA/LABACOLLA'

In [137]:
dias = [8,9,10,11,12]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1505') &
                                                    (AEMET_Realidad['LONGITUD'] == 727272) &
                                                    (AEMET_Realidad['LATITUD'] == 430640) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

dias1 = [9,10,11,12,13]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1505') &
                                                    (AEMET_Realidad['LONGITUD'] == 727272) &
                                                    (AEMET_Realidad['LATITUD'] == 430640) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 8) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

In [138]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '1505', 'LATITUD'] = 430641

In [139]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1549') &
                                                    (AEMET_Realidad['LATITUD'] == 425600) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 5)].index)

In [140]:
dias = [9,10,11,12,13,14]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '2422') &
                                                    (AEMET_Realidad['LONGITUD'] == 445152) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 8) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

dias1 = [8,9,10,11,12]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '2422') &
                                                    (AEMET_Realidad['LONGITUD'] == 445152) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

In [141]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '2422', 'LONGITUD'] = 445162

In [142]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '3129') &
                                                    (AEMET_Realidad['LONGITUD'] == 333202) &
                                                    (AEMET_Realidad['LATITUD'].isnull())].index)

In [143]:
dias = [2,3,4,5,6]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '3195') &
                                                    (AEMET_Realidad['NOMBRE'] == 'MADRID, RETIRO')&
                                                    (AEMET_Realidad['ANYO'] == 2018) &
                                                    (AEMET_Realidad['MES'] == 6) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

dias1 = [10,11,12,13,14]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '3195') &
                                                    (AEMET_Realidad['NOMBRE'] == 'MADRID, RETIRO')&
                                                    (AEMET_Realidad['ANYO'] == 2020) &
                                                    (AEMET_Realidad['MES'] == 2) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '3195', 'NOMBRE'] = 'MADRID, RETIRO'

In [144]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '4121') &
                                                    (AEMET_Realidad['LONGITUD'] == 355132) &
                                                    (AEMET_Realidad['LATITUD'] == 389900)].index)

In [145]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '5402') &
                                                    (AEMET_Realidad['DIA'] == 27) &
                                                    (AEMET_Realidad['MES'] == 2) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['LONGITUD'] == 450462) &
                                                    (AEMET_Realidad['LATITUD'] == 375039)].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '5402', ['LATITUD', 'LONGITUD']] = [375056, 450482]

In [146]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '5973', 'NOMBRE'] = 'CADIZ,OBS.'

In [147]:
dias1 = [9,10,11,12,13,14]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9170') &
                                                    (AEMET_Realidad['NOMBRE'] == 'LOGROÑO, AEROPUERTO')&
                                                    (AEMET_Realidad['ANYO'] == 2021) &
                                                    (AEMET_Realidad['MES'] == 4) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '9170', 'NOMBRE'] = 'LOGROÑO, AEROPUERTO'

In [148]:
dias = [29,30]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9898') &
                                                    (AEMET_Realidad['LONGITUD'] == 19322) &
                                                    (AEMET_Realidad['LATITUD'] == 420504) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 9) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

dias1 = [1,2,3]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9898') &
                                                    (AEMET_Realidad['LONGITUD'] == 19322) &
                                                    (AEMET_Realidad['LATITUD'] == 420504) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 10) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

dias2 = [9,10,11,12]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9898') &
                                                    (AEMET_Realidad['LONGITUD'] == 19322) &
                                                    (AEMET_Realidad['LATITUD'] == 420504) &
                                                    (AEMET_Realidad['ANYO'] == 2021) &
                                                    (AEMET_Realidad['MES'] == 4) &
                                                    (AEMET_Realidad['DIA'].isin(dias2))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '9898', 'NOMBRE'] = 'HUESCA, AEROPUERTO'

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '9898', ['LATITUD', 'LONGITUD']] = [420504, 19322]

In [149]:
dias = [22,23,24,25,26]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '0016A') &
                                                    (AEMET_Realidad['LONGITUD'] == 109491) &
                                                    (AEMET_Realidad['LATITUD'] == 410842) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 9) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '0016A', ['LATITUD', 'LONGITUD']] = [410842, 109491]

In [150]:
dias1 = [11,12,13,14,15]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1024E') &
                                                    (AEMET_Realidad['NOMBRE'] == 'SAN SEBASTIÁN,IGUELDO')&
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 5) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '1024E', 'NOMBRE'] = 'SAN SEBASTIÁN,IGUELDO'

In [151]:
dias = [8,9,10,11,12,13]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1249X') &
                                                    (AEMET_Realidad['LONGITUD'] == 552262) &
                                                    (AEMET_Realidad['LATITUD'] == 432112) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

dias1 = [9,10,11,12,13]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1249X') &
                                                    (AEMET_Realidad['LONGITUD'] == 552262) &
                                                    (AEMET_Realidad['LATITUD'] == 432112) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 8) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '1249X', ['LATITUD', 'LONGITUD']] = [432112, 552262]

In [152]:
dias = [8,9,10,11,12,13]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '3168D') &
                                                    (AEMET_Realidad['LONGITUD'] == 309012) &
                                                    (AEMET_Realidad['LATITUD'] == 403749) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

dias1 = [9,10,11,12,13]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '3168D') &
                                                    (AEMET_Realidad['LONGITUD'] == 309012) &
                                                    (AEMET_Realidad['LATITUD'] == 403749) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 8) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '3168D', ['LATITUD', 'LONGITUD']] = [403749, 309002]

In [153]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '3260B') &
                                                    (AEMET_Realidad['LONGITUD'] == 402432) &
                                                    (AEMET_Realidad['LATITUD'] == 398800)].index)

In [154]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '4642E', ['LATITUD', 'LONGITUD']] = [371642, 654422]

In [155]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '5270B', ['LONGITUD', 'LATITUD']] = [348322, 374639]

In [156]:
dias = [2,3,4,5,6]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '5530E') &
                                                    (AEMET_Realidad['LONGITUD'] == 347232) &
                                                    (AEMET_Realidad['LATITUD'] == 371125) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 2) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '5530E', ['LATITUD', 'LONGITUD']] = [371125, 347232]

In [157]:
dias = [26,27,28]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '5790Y') &
                                                    (AEMET_Realidad['LONGITUD'] == 600162) &
                                                    (AEMET_Realidad['LATITUD'] == 372158) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['MES'] == 2) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '5790Y', ['LATITUD', 'LONGITUD']] = [372151, 600212]

In [158]:
dias = [8,9,10,11,12]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '8368U') &
                                                    (AEMET_Realidad['LONGITUD'] == 107272) &
                                                    (AEMET_Realidad['LATITUD'] == 402103) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7) &
                                                    (AEMET_Realidad['DIA'].isin(dias))].index)

dias1 = [9,10,11,12,13]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '8368U') &
                                                    (AEMET_Realidad['LONGITUD'] == 107272) &
                                                    (AEMET_Realidad['LATITUD'] == 402103) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 8) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '8368U', ['LATITUD', 'LONGITUD']] = [402102, 107272]

In [159]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '8500A', ['LATITUD', 'LONGITUD']] = [395726, 4192]
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '8500A', 'NOMBRE'] = 'CASTELLÓN-ALMAZORA'

In [160]:
dias1 = [28,29,30]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9091R') &
                                                    (AEMET_Realidad['LONGITUD'] == 244002) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['MES'] == 11) &
                                                    (AEMET_Realidad['DIA'].isin(dias1))].index)

dias2 = [1,2]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9091R') &
                                                    (AEMET_Realidad['LONGITUD'] == 244002) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['MES'] == 12) &
                                                    (AEMET_Realidad['DIA'].isin(dias2))].index)

dias3 = [15,16,17,18,19,20]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9091R') &
                                                    (AEMET_Realidad['LONGITUD'] == 243592) &
                                                    (AEMET_Realidad['ANYO'] == 2018) &
                                                    (AEMET_Realidad['MES'] == 5) &
                                                    (AEMET_Realidad['DIA'].isin(dias3))].index)

dias5 = [11,12,13,14,15]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9091R') &
                                                    (AEMET_Realidad['NOMBRE'] == 'VITORIA-GASTEIZ AEROPUERTO') &
                                                    (AEMET_Realidad['LONGITUD'] == 243582) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 5) &
                                                    (AEMET_Realidad['DIA'].isin(dias5))].index)

In [161]:
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '9091R', ['LATITUD', 'LONGITUD']] = [425219, 243582]
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '9091R', 'NOMBRE'] = 'VITORIA-GASTEIZ AEROPUERTO'

In [162]:
dias5 = [9,10,11,12,13]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9263D') &
                                                    (AEMET_Realidad['NOMBRE'] == 'PAMPLONA, AEROPUERTO') &
                                                    (AEMET_Realidad['ANYO'] == 2021) &
                                                    (AEMET_Realidad['MES'] == 4) &
                                                    (AEMET_Realidad['DIA'].isin(dias5))].index)


AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '9263D', 'NOMBRE'] = 'PAMPLONA, AEROPUERTO'

In [163]:
dias5 = [12,13,14,15,16]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9434P') &
                                                    (AEMET_Realidad['NOMBRE'] == 'ZARAGOZA, VALDESPARTERA') &
                                                    (AEMET_Realidad['LONGITUD'] == 56062)&
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 9) &
                                                    (AEMET_Realidad['DIA'].isin(dias5))].index)

dias6 = [9,10,11,12,13]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '9434P') &
                                                    (AEMET_Realidad['NOMBRE'] == 'ZARAGOZA-VALDESPARTERA') &
                                                    (AEMET_Realidad['ANYO'] == 2021) &
                                                    (AEMET_Realidad['MES'] == 4) &
                                                    (AEMET_Realidad['DIA'].isin(dias6))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '9434P', ['LATITUD', 'LONGITUD']] = [413716, 56052]
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == '9434P', 'NOMBRE'] = 'ZARAGOZA, VALDESPARTERA'

In [164]:
dias6 = [2,3,4,5,6]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == 'B228') &
                                                    (AEMET_Realidad['NOMBRE'] == 'PALMA-PUERTO') &
                                                    (AEMET_Realidad['ANYO'] == 2020) &
                                                    (AEMET_Realidad['MES'] == 10) &
                                                    (AEMET_Realidad['DIA'].isin(dias6))].index)

AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == 'B228', ['LATITUD', 'LONGITUD']] = [393319, 237311]
AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == 'B228', 'NOMBRE'] = 'PALMA-PUERTO'

In [165]:
dias6 = [1,2,3,4]

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == 'C649I') &
                                                    (AEMET_Realidad['LONGITUD'] == 1523222) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 10) &
                                                    (AEMET_Realidad['DIA'].isin(dias6))].index)

dias7 = [29,30]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == 'C649I') &
                                                    (AEMET_Realidad['LONGITUD'] == 1523222) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 9) &
                                                    (AEMET_Realidad['DIA'].isin(dias7))].index)


AEMET_Realidad.loc[AEMET_Realidad['INDICATIVO'] == 'C649I', ['LATITUD', 'LONGITUD']] = [275504, 1523432]

In [166]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['INDICATIVO'] == '1495') &
                                                    (AEMET_Realidad['LONGITUD'] == 837262) &
                                                    (AEMET_Realidad['LATITUD'].isnull())].index)

________________________________________________________________________________

In [167]:
AEMET_Realidad = AEMET_Realidad.drop_duplicates()

In [168]:
dia = [26,27,28,29,30,31]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'REUS/AEROPUERTO') &
                                                    (AEMET_Realidad['ALTITUD'] == 68) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['MES'] == 1) &
                                                    (AEMET_Realidad['DIA'].isin(dia))].index)

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'REUS/AEROPUERTO') &
                                                    (AEMET_Realidad['ALTITUD'] == 68) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['MES'] == 2) &
                                                    (AEMET_Realidad['DIA'] == 1)].index)

In [169]:
dia = [26,27,28]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'TARRAGONA, FAC. GEOGRAFÍA') &
                                                    (AEMET_Realidad['ALTITUD'] == 55) &
                                                    (AEMET_Realidad['ANYO'] == 2015)&
                                                    (AEMET_Realidad['MES'] == 1) &
                                                    (AEMET_Realidad['DIA'].isin(dia))].index)

In [170]:
dia = [26,27,28,29,30,31]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'BARCELONA/AEROPUERTO') &
                                                    (AEMET_Realidad['ALTITUD'] == 4) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['MES'] == 1) &
                                                    (AEMET_Realidad['DIA'].isin(dia))].index)

dia1 = [1,2]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'BARCELONA/AEROPUERTO') &
                                                    (AEMET_Realidad['ALTITUD'] == 4) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['MES'] == 2) &
                                                    (AEMET_Realidad['DIA'].isin(dia1))].index)

In [171]:
dia = [26,27,28]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'BARCELONA,CMT') &
                                                    (AEMET_Realidad['ALTITUD'] == 25) &
                                                    (AEMET_Realidad['ANYO'] == 2015) &
                                                    (AEMET_Realidad['MES'] == 1) &
                                                    (AEMET_Realidad['DIA'].isin(dia))].index)

In [172]:
dia = [9,10,11,12,13]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'SAN SEBASTIÁN,IGUELDO') &
                                                    (AEMET_Realidad['ALTITUD'] == 250) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 8) &
                                                    (AEMET_Realidad['DIA'].isin(dia))].index)

In [173]:
dia = [8,9,10,11,12,13]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'SANTANDER,CMT') &
                                                    (AEMET_Realidad['ALTITUD'] == 51) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7) &
                                                    (AEMET_Realidad['DIA'].isin(dia))].index)

In [174]:
dia = [7,8,9,10,11]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'OVIEDO') &
                                                    (AEMET_Realidad['ALTITUD'] == 338) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 2) &
                                                    (AEMET_Realidad['DIA'].isin(dia))].index)

In [175]:
mes = [6,7]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'A CORUÑA') &
                                                    (AEMET_Realidad['ALTITUD'] == 57) &
                                                    (AEMET_Realidad['MES'].isin(mes)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MAX'].isnull())].index)

dia = [8,9,10,11,12,13]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'A CORUÑA') &
                                                    (AEMET_Realidad['ALTITUD'] == 57) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [176]:
mes = [10,11]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'VIGO/PEINADOR') &
                                                    (AEMET_Realidad['ALTITUD'] == 255) &
                                                    (AEMET_Realidad['MES'].isin(mes)) &
                                                    (AEMET_Realidad['ANYO'] == 2022) &
                                                    (AEMET_Realidad['MAX'].isnull())].index)

In [177]:
mes = [6,7]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'PONFERRADA') &
                                                    (AEMET_Realidad['ALTITUD'] == 534) &
                                                    (AEMET_Realidad['MES'].isin(mes)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MAX'].isnull())].index)

In [178]:
mes = [6,7]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'TOLEDO') &
                                                    (AEMET_Realidad['ALTITUD'] == 513) &
                                                    (AEMET_Realidad['MES'].isin(mes)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MAX'].isnull())].index)

dia = [8,9,10,11,12]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'TOLEDO') &
                                                    (AEMET_Realidad['ALTITUD'] == 513) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [179]:
dia = [8,9,10,11,12,13]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'PONTEVEDRA') &
                                                    (AEMET_Realidad['ALTITUD'] == 113) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [180]:
dia = [8,9,10,11,12,13]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'OURENSE') &
                                                    (AEMET_Realidad['ALTITUD'] == 143) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [181]:
dia = [8,9,10,11,12,13]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'SORIA') &
                                                    (AEMET_Realidad['ALTITUD'] == 1081) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [182]:
dia = [8,9,10,11,12]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'SEGOVIA') &
                                                    (AEMET_Realidad['ALTITUD'] == 1005) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [183]:
mes = [6,7]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'CIUDAD REAL') &
                                                    (AEMET_Realidad['ALTITUD'] == 626) &
                                                    (AEMET_Realidad['MES'].isin(mes)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MAX'].isnull())].index)

dia = [8,9,10,11,12]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'CIUDAD REAL') &
                                                    (AEMET_Realidad['ALTITUD'] == 626) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [184]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'VALLADOLID') &
                                                    (AEMET_Realidad['ALTITUD'] == 734) &
                                                    (AEMET_Realidad['DIA'] == 8) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [185]:
dia = [8,9,10,11,12]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'BADAJOZ/TALAVERA LA REAL') &
                                                    (AEMET_Realidad['ALTITUD'] == 185) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

dia = [25,26,27,28,29,30]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'BADAJOZ/TALAVERA LA REAL') &
                                                    (AEMET_Realidad['ALTITUD'] == 185) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 11)].index)

In [186]:
dia = [14,15,16,17,18]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'HUELVA, RONDA ESTE') &
                                                    (AEMET_Realidad['ALTITUD'] == 18) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2022) &
                                                    (AEMET_Realidad['MES'] == 10)].index)

In [187]:
dia = [25,26,27,28,29,30]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'GRANADA/AEROPUERTO') &
                                                    (AEMET_Realidad['ALTITUD'] == 560) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 11)].index)

In [188]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'CADIZ,OBS.') &
                                                    (AEMET_Realidad['ALTITUD'] == 2) &
                                                    (AEMET_Realidad['DIA'] == 1) &
                                                    (AEMET_Realidad['ANYO'] == 2011) &
                                                    (AEMET_Realidad['MAX'].isnull()) &
                                                    (AEMET_Realidad['MES'] == 3)].index)

In [189]:
dia = [8,9,10,11,12]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'MURCIA') &
                                                    (AEMET_Realidad['ALTITUD'] == 61) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [190]:
dia = [8,9,10,11,12]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'CUENCA') &
                                                    (AEMET_Realidad['ALTITUD'] == 948) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [191]:
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'ALBACETE,OBS.') &
                                                    (AEMET_Realidad['ALTITUD'] == 676) &
                                                    (AEMET_Realidad['MES'] == 6) &
                                                    (AEMET_Realidad['ANYO'] == 2018) &
                                                    (AEMET_Realidad['MAX'].isnull())].index)

In [192]:
ANIO = [2009,2010,2011]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'VITORIA-GASTEIZ AEROPUERTO') &
                                                    (AEMET_Realidad['ALTITUD'] == 513) &
                                                    (AEMET_Realidad['MES'] == 3) &
                                                    (AEMET_Realidad['DIA'] == 1) &
                                                    (AEMET_Realidad['ANYO'].isin(ANIO)) &
                                                    (AEMET_Realidad['MAX'].isnull())].index)

In [193]:
dia = [14,15,16,17,18,19]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'ALBACETE,OBS.') &
                                                    (AEMET_Realidad['ALTITUD'] == 676) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2018) &
                                                    (AEMET_Realidad['MES'] == 6)].index)

dia = [15,16,17,18,19]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'ALBACETE,OBS.') &
                                                    (AEMET_Realidad['ALTITUD'] == 676) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2022) &
                                                    (AEMET_Realidad['MES'] == 10)].index)

In [194]:
dia = [27,28,29,30]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'CASTELLÓN-ALMAZORA') &
                                                    (AEMET_Realidad['ALTITUD'] == 43) &
                                                    (AEMET_Realidad['DIA'].isin(dia)) &
                                                    (AEMET_Realidad['ANYO'] == 2022) &
                                                    (AEMET_Realidad['MES'] == 4)].index)

dia1 = [8,9,10,11,12]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'CASTELLÓN-ALMAZORA') &
                                                    (AEMET_Realidad['ALTITUD'] == 43) &
                                                    (AEMET_Realidad['DIA'].isin(dia1)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'CASTELLÓN-ALMAZORA') &
                                                    (AEMET_Realidad['ALTITUD'] == 43) &
                                                    (AEMET_Realidad['DIA'] == 1) &
                                                    (AEMET_Realidad['ANYO'] == 2022) &
                                                    (AEMET_Realidad['MES'] == 5)].index)

In [195]:
dia1 = [8,9,10,11,12]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'LLEIDA') &
                                                    (AEMET_Realidad['ALTITUD'] == 185) &
                                                    (AEMET_Realidad['DIA'].isin(dia1)) &
                                                    (AEMET_Realidad['ANYO'] == 2023) &
                                                    (AEMET_Realidad['MES'] == 7)].index)

In [196]:
dia1 = [1,2,3,4,5]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'HUESCA, AEROPUERTO') &
                                                    (AEMET_Realidad['ALTITUD'] == 541) &
                                                    (AEMET_Realidad['DIA'].isin(dia1)) &
                                                    (AEMET_Realidad['ANYO'] == 2016) &
                                                    (AEMET_Realidad['MES'] == 11)].index)

In [197]:
dia1 = [9,10,11,12,13,14]
AEMET_Realidad = AEMET_Realidad.drop(AEMET_Realidad[(AEMET_Realidad['NOMBRE'] == 'GRAN CANARIA/AEROPUERTO') &
                                                    (AEMET_Realidad['ALTITUD'] == 24) &
                                                    (AEMET_Realidad['DIA'].isin(dia1)) &
                                                    (AEMET_Realidad['ANYO'] == 2022) &
                                                    (AEMET_Realidad['MES'] == 11)].index)

In [198]:
# Calcular el porcentaje de valores vacíos por columna
porcentaje_perdidos = AEMET_Realidad.isnull().mean() * 100
porcentaje_perdidos_act = AEMET_Realidad.isnull().mean() * 100

# Mostrar el porcentaje de valores vacíos por columna
print(porcentaje_perdidos)
print()
print(porcentaje_perdidos_act)

INDICATIVO    0.000000
ANYO          0.000000
MES           0.000000
DIA           0.000000
NOMBRE        0.000000
ALTITUD       0.654437
NOM_PROV      0.654437
LONGITUD      0.000000
LATITUD       0.000000
MAX           5.668795
MIN           6.099409
FECHA         0.000000
dtype: float64

INDICATIVO    0.000000
ANYO          0.000000
MES           0.000000
DIA           0.000000
NOMBRE        0.000000
ALTITUD       0.654437
NOM_PROV      0.654437
LONGITUD      0.000000
LATITUD       0.000000
MAX           5.668795
MIN           6.099409
FECHA         0.000000
dtype: float64


In [199]:
AEMET_Realidad.to_csv(path + 'AEMET_Realidad_limpio.csv', index=False)

In [200]:
Prediccion_AEMET_corta = AEMET_Previsiones[AEMET_Previsiones['FECHA_FICH'] == AEMET_Previsiones['FECHA']]
Prediccion_AEMET_corta['FECHA'] = pd.to_datetime(Prediccion_AEMET_corta['FECHA'], format='%d/%m/%Y')

print("DataFrame Filtrado:")
print(Prediccion_AEMET_corta.head(15))

DataFrame Filtrado:
     FECHA_FICH  HORA_FICH      FECHA                  DESC_ESTACION  \
0    03/12/2014          0 2014-12-03                       A Coruña   
10   03/12/2014          0 2014-12-03                       Albacete   
20   03/12/2014          0 2014-12-03            Albacete Base Aérea   
30   03/12/2014          0 2014-12-03               Alicante/Alacant   
40   03/12/2014          0 2014-12-03             Almería Aeropuerto   
50   03/12/2014          0 2014-12-03               Autilla del Pino   
60   03/12/2014          0 2014-12-03             Badajoz Aeropuerto   
70   03/12/2014          0 2014-12-03                      Barcelona   
80   03/12/2014          0 2014-12-03           Barcelona Aeropuerto   
90   03/12/2014          0 2014-12-03              Bilbao Aeropuerto   
100  03/12/2014          0 2014-12-03              Burgos Aeropuerto   
110  03/12/2014          0 2014-12-03  Castellón de la Plana, Almazo   
120  03/12/2014          0 2014-12-03       

C:\Users\joset\AppData\Local\Temp\ipykernel_42688\3205320432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Prediccion_AEMET_corta['FECHA'] = pd.to_datetime(Prediccion_AEMET_corta['FECHA'], format='%d/%m/%Y')


In [201]:
# Hacer la combinación de los DataFrames
merged_df = AEMET_Realidad.merge(Prediccion_AEMET_corta, left_on=['FECHA', 'INDICATIVO'], right_on=['FECHA', 'ID_ESTACION'], how='left')

# Rellenar los valores perdidos en MAX y MIN
merged_df['MAX'] = merged_df['MAX'].combine_first(merged_df['T_MAX'])
merged_df['MIN'] = merged_df['MIN'].combine_first(merged_df['T_MIN'])

# Seleccionar solo las columnas originales de AEMET_Realidad
AEMET_Realidad_actualizado = merged_df[AEMET_Realidad.columns]

print("DataFrame AEMET_Realidad Actualizado:")
print(AEMET_Realidad_actualizado)

DataFrame AEMET_Realidad Actualizado:
       INDICATIVO  ANYO  MES  DIA                   NOMBRE  ALTITUD  \
0           0016A  2009    1    1          REUS/AEROPUERTO     71.0   
1           0016A  2009    1    2          REUS/AEROPUERTO     71.0   
2           0016A  2009    1    3          REUS/AEROPUERTO     71.0   
3           0016A  2009    1    4          REUS/AEROPUERTO     71.0   
4           0016A  2009    1    5          REUS/AEROPUERTO     71.0   
...           ...   ...  ...  ...                      ...      ...   
334634      C649I  2024    6   26  GRAN CANARIA/AEROPUERTO     24.0   
334635      C649I  2024    6   27  GRAN CANARIA/AEROPUERTO     24.0   
334636      C649I  2024    6   28  GRAN CANARIA/AEROPUERTO     24.0   
334637      C649I  2024    6   29  GRAN CANARIA/AEROPUERTO     24.0   
334638      C649I  2024    6   30  GRAN CANARIA/AEROPUERTO     24.0   

          NOM_PROV   LONGITUD   LATITUD   MAX   MIN      FECHA  
0        TARRAGONA   109491.0  410842.0  13.

In [202]:
#creamos una funcion para colocar el signo correcto a las Longitudes y Latitudes
def ajustar_longitud(df):
    df=df.copy()
    # Diccionario con provincias y si están al oeste del meridiano
    provincias_oeste = {
        'TARRAGONA': False, 'BARCELONA': False, 'GIRONA': False, 'GIPUZKOA': True,
        'BIZKAIA': True, 'CANTABRIA': True, 'ASTURIAS': True, 'A CORUÑA': True,
        'PONTEVEDRA': True, 'LUGO': True, 'LEON': True, 'OURENSE': True,
        'SORIA': True, 'BURGOS': True, 'PALENCIA': True, 'VALLADOLID': True,
        'AVILA': True, 'SEGOVIA': True, 'ZAMORA': True, 'SALAMANCA': True,
        'MADRID': True, 'GUADALAJARA': True, 'TOLEDO': True, 'CACERES': True,
        'CIUDAD REAL': True, 'BADAJOZ': True, 'HUELVA': True, 'JAEN': True,
        'CORDOBA': True, 'GRANADA': True, 'SEVILLA': True, 'CADIZ': True,
        'MALAGA': True, 'ALMERIA': True, 'MURCIA': False, 'ALICANTE': False,
        'CUENCA': True, 'ALBACETE': True, 'TERUEL': False, 'VALENCIA': False,
        'CASTELLON': False, 'ARABA/ALAVA': True, 'LA RIOJA': True, 'NAVARRA': True,
        'ZARAGOZA': False, 'LLEIDA': False, 'HUESCA': False, 'BALEARES': False,
        'LAS PALMAS': True
    }
    
    # Aplicar el ajuste de longitud
    df['LONGITUD'] = df.apply(lambda row: -row['LONGITUD'] if provincias_oeste.get(row['NOM_PROV'], False) else row['LONGITUD'], axis=1)
    return df

In [203]:
#creamos una funcion para corregir las posiciones de las latitudes y longitudes
def ajustar_formato_longitud_latitud(df):
    df=df.copy()
    def convertir_longitud(valor):
        if pd.isnull(valor) or not isinstance(valor, (int, float, str)):
            return None
        try:
            valor_str = str(valor).strip().replace('.', '')
            if len(valor_str) < 6:
                return None
            parte_entera = valor_str[:-6]
            parte_decimal = valor_str[-6:]
            return float(f"{parte_entera}.{parte_decimal}")
        except ValueError:
            return None

    def convertir_latitud(valor):
        if pd.isnull(valor) or not isinstance(valor, (int, float, str)):
            return None
        try:
            valor_str = str(valor).strip().replace('.', '')
            if len(valor_str) < 6:
                return None
            parte_entera = valor_str[:-5]
            parte_decimal = valor_str[-5:]
            return float(f"{parte_entera}.{parte_decimal}")
        except ValueError:
            return None

    df['LONGITUD'] = df['LONGITUD'].apply(convertir_longitud)
    df['LATITUD'] = df['LATITUD'].apply(convertir_latitud)
    
    return df 

In [204]:
#creamos un formato mas amigable para las busquedas por coordinadas
def convertir_a_formato_busqueda(df):
    df=df.copy()
    # Crear la columna 'GOOGLE_MAPS' con el formato 'latitud,longitud'
    df['GOOGLE_MAPS'] = df.apply(lambda row: f"{row['LATITUD']},{row['LONGITUD']}", axis=1)
    return df

In [205]:
AEMET_Realidad_actualizado = ajustar_longitud(AEMET_Realidad_actualizado)
AEMET_Realidad_actualizado = ajustar_formato_longitud_latitud(AEMET_Realidad_actualizado)
AEMET_Realidad_actualizado = convertir_a_formato_busqueda(AEMET_Realidad_actualizado)
AEMET_Realidad_actualizado.head()

,INDICATIVO,ANYO,MES,DIA,NOMBRE,ALTITUD,NOM_PROV,LONGITUD,LATITUD,MAX,MIN,FECHA,GOOGLE_MAPS
0,0016A,2009,1,1,REUS/AEROPUERTO,71.0,TARRAGONA,1.09491,41.0842,13.0,4.2,2009-01-01,"41.0842,1.09491"
1,0016A,2009,1,2,REUS/AEROPUERTO,71.0,TARRAGONA,1.09491,41.0842,11.3,4.2,2009-01-02,"41.0842,1.09491"
2,0016A,2009,1,3,REUS/AEROPUERTO,71.0,TARRAGONA,1.09491,41.0842,10.6,6.1,2009-01-03,"41.0842,1.09491"
3,0016A,2009,1,4,REUS/AEROPUERTO,71.0,TARRAGONA,1.09491,41.0842,11.3,-0.8,2009-01-04,"41.0842,1.09491"
4,0016A,2009,1,5,REUS/AEROPUERTO,71.0,TARRAGONA,1.09491,41.0842,10.6,-3.1,2009-01-05,"41.0842,1.09491"


In [206]:
AEMET_Realidad_actualizado.to_csv(path + '\\Cleaning\\AEMET_Realidad_clean.csv', index=False)

In [207]:
# Calcular el porcentaje de valores vacíos por columna
porcentaje_perdidos = AEMET_Realidad_actualizado.isnull().mean() * 100
porcentaje_perdidos_act = AEMET_Realidad_actualizado.isnull().mean() * 100

# Mostrar el porcentaje de valores vacíos por columna
print(porcentaje_perdidos)
print()
print(porcentaje_perdidos_act)

INDICATIVO     0.000000
ANYO           0.000000
MES            0.000000
DIA            0.000000
NOMBRE         0.000000
ALTITUD        0.654437
NOM_PROV       0.654437
LONGITUD       1.683008
LATITUD        0.000000
MAX            3.583563
MIN            3.584161
FECHA          0.000000
GOOGLE_MAPS    0.000000
dtype: float64

INDICATIVO     0.000000
ANYO           0.000000
MES            0.000000
DIA            0.000000
NOMBRE         0.000000
ALTITUD        0.654437
NOM_PROV       0.654437
LONGITUD       1.683008
LATITUD        0.000000
MAX            3.583563
MIN            3.584161
FECHA          0.000000
GOOGLE_MAPS    0.000000
dtype: float64


In [208]:
def calcular_temperaturas_promedio_por_provincia_y_dia(df):
    """
    Calcula las temperaturas máximas y mínimas por día, promediando por provincia.

    Parámetros:
    df (pandas.DataFrame): El DataFrame que contiene las columnas 'FECHA', 'NOM_PROV', 'MAX', y 'MIN'.

    Retorna:
    pandas.DataFrame: Un DataFrame con las temperaturas promedio por provincia y día.
    """
    # Asegurarse de que la columna FECHA esté en formato de fecha
    df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d')

    # Agrupar por NOM_PROV y FECHA para calcular el promedio de MAX y MIN
    temperatura_promedio = df.groupby(['NOM_PROV', 'FECHA']).agg({
        'MAX': 'mean',
        'MIN': 'mean'
    }).reset_index()

    # Redondear las temperaturas a dos decimales
    temperatura_promedio['MAX'] = temperatura_promedio['MAX'].round(2)
    temperatura_promedio['MIN'] = temperatura_promedio['MIN'].round(2)

    return temperatura_promedio

In [209]:
# Uso de la función con el DataFrame AEMET_Realidad_actualizado
provincias_temperaturas_promedio = calcular_temperaturas_promedio_por_provincia_y_dia(AEMET_Realidad_actualizado)
# Ordenar el DataFrame por la columna FECHA
provincias_temperaturas_promedio = provincias_temperaturas_promedio.sort_values(by='FECHA')
# Verificar el resultado
provincias_temperaturas_promedio.head(3)

,NOM_PROV,FECHA,MAX,MIN
0,A CORUÑA,2009-01-01,15.35,10.8
202455,PALENCIA,2009-01-01,9.00,2.5
50799,BARCELONA,2009-01-01,13.40,7.9


In [210]:
#Guardamos nuestro dataset por si lo necesitamos mas adelante
provincias_temperaturas_promedio.to_csv(path + '\\Cleaning\\provincias_temperaturas_promedio.csv', index=False)